In [ ]:
import tensorflow as tf
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Activation, Flatten, BatchNormalization
from keras.layers import Conv2D, MaxPooling2D, Input, AveragePooling2D
import keras
from keras.datasets import cifar10
import matplotlib
import matplotlib.pyplot as plt
from keras.preprocessing.image import ImageDataGenerator
from keras.regularizers import l2
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, ReduceLROnPlateau

In [ ]:
# 搭建基本模块
def resnet_block(inputs, num_filters=16,kernel_size=3,strides=1, activation='relu',weight_decay=0.0005):
    x = Conv2D(num_filters, kernel_size=kernel_size, strides=strides, padding='same',kernel_initializer='he_normal', kernel_regularizer=regularizers.l2(weight_decay))(inputs)
    x = BatchNormalization()(x)
    if (activation):
        x = Activation('relu')(x)
    return x

# 建一个20层的ResNet网络 
def resnet(input_shape):
    inputs = Input(shape=input_shape)
    
    #第一层
    x = resnet_block(inputs)

    # 第2~7层
    for i in range(6):
        a = resnet_block(inputs = x)
        b = resnet_block(inputs=a,activation=None)
        x = keras.layers.add([x,b])
        x = Activation('relu')(x)
        
    # 第8~13层
    for i in range(6):
        if i == 0:
            a = resnet_block(inputs = x,strides=2,num_filters=32)
        else:
            a = resnet_block(inputs = x,num_filters=32)
        b = resnet_block(inputs=a,activation=None,num_filters=32)
        if i==0:
            x = Conv2D(32,kernel_size=3,strides=2,padding='same',
                       kernel_initializer='he_normal',kernel_regularizer=l2(1e-4))(x)
        x = keras.layers.add([x,b])
        x = Activation('relu')(x)

    # 第14~19层
    for i in range(6):
        if i ==0 :
            a = resnet_block(inputs = x,strides=2,num_filters=64)
        else:
            a = resnet_block(inputs = x,num_filters=64)

        b = resnet_block(inputs=a,activation=None,num_filters=64)
        if i == 0:
            x = Conv2D(64,kernel_size=3,strides=2,padding='same',
                       kernel_initializer='he_normal',kernel_regularizer=l2(1e-4))(x)
        x = keras.layers.add([x,b])
        x = Activation('relu')(x)

    # 第20层   
    x = AveragePooling2D(pool_size=2)(x)
    y = Flatten()(x)
    outputs = Dense(10,activation='softmax',
                    kernel_initializer='he_normal')(y)
    
    #初始化模型，之前的操作只是将多个神经网络层进行了相连，通过下面这一句的初始化操作，才算真正完成了一个模型的结构初始化
    model = Model(inputs=inputs,outputs=outputs)
    return model

    #标准化
def standardization(x_train,x_test):      
    mean = np.mean(x_train,axis=(0,1,2,3))
    std = np.std(x_train, axis=(0, 1, 2, 3))
    x_train = (x_train-mean)/(std+1e-7)
    x_test = (x_test-mean)/(std+1e-7)
    return x_train, x_test

#归一化
def normalize(x_train,x_test):
    x_train_max=np.max(x_train)
    x_train_min=np.min(x_train)
    x_test_max=np.max(x_test)
    x_test_min=np.min(x_test)
    x_train=(x_train-x_train_min)/(x_train_max-x_train_min)
    x_test=(x_test-x_test_min)/(x_test_max-x_test_min)
    return x_train, x_test
   

In [ ]:
#训练参数
batch_size = 128
maxepoches = 100
learning_rate = 0.01
lr_decay = 1e-6
lr_drop = 20
num_classes = 10
weight_decay = 0.0005
x_shape = [32,32,3]

# 数据载入
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

# 多分类标签生成
y_train = keras.utils.to_categorical(y_train, classes_num)
y_test = keras.utils.to_categorical(y_test, classes_num)
# 生成训练数据
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
#调用归一化
x_train, x_test = normalize(x_train, x_test)

#动态学习率
'''
def lr_scheduler(epoch):
    return learning_rate * (0.5 ** (epoch // lr_drop))
reduce_lr = keras.callbacks.LearningRateScheduler(lr_scheduler)
'''

#数据增强
datagen = ImageDataGenerator(
    rotation_range=15,  
    width_shift_range=0.1, 
    height_shift_range=0.1, 
    horizontal_flip=True)       
datagen.fit(x_train)

#模型搭建
model = resnet(x_shape)
#优化器设置
sgd = optimizers.SGD(lr=learning_rate, decay=lr_decay, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd,metrics=['accuracy'])
#model.compile(loss='categorical_crossentropy', optimizer=Adam(), metrics=['accuracy'])

#回调函数设置
checkpoint = ModelCheckpoint(filepath='./resnet.h5', monitor='val_acc', verbose=1,save_best_only=True)
#动态学习率
lr_scheduler = LearningRateScheduler(lr_sch)
lr_reducer = ReduceLROnPlateau(monitor='val_acc', factor=0.2, patience=5, mode='max', min_lr=1e-3)
callbacks = [checkpoint, lr_scheduler, lr_reducer]

In [ ]:
#模型训练
hist = model.fit(x_train, y_train, batch_size=batch_size, steps_per_epoch=x_train.shape[0] // batch_size,epochs=maxepoches, validation_data=(x_test,y_test), verbose=1,callbacks=callbacks)
#hist = model.fit_generator(datagen.flow(x_train, y_train, batch_size=batch_size), steps_per_epoch=x_train.shape[0] // batch_size, epochs = maxepoches, validation_data=(x_test,y_test), verbose=1, callbacks=callbacks)


In [ ]:
#输出准确率
scores = model.evaluate(x_test,y_test,verbose=1)
print('Test loss:',scores[0])
print('Test accuracy:',scores[1])

# 绘图
train_acc = hist.history['accuracy']
val_acc = hist.history['val_accuracy']
train_loss = hist.history['loss']
val_loss = hist.history['val_loss']

epochs = range(1, len(train_acc)+1)
plt.plot(epochs, train_acc, 'bo', label = 'Training acc')
plt.plot(epochs, val_acc, 'r', label = 'Validation acc')
plt.title('Training and validation accuracy')
plt.legend()
plt.savefig("vgg_accuracy.png")
plt.figure() 
plt.plot(epochs, train_loss, 'bo', label = 'Training loss')
plt.plot(epochs, val_loss, 'r', label = 'Validation loss')
plt.title('Training and validation loss')
plt.legend()
plt.savefig("vgg_loss.png")